## Adding libraries.

In [18]:
options(warn = -1)
library(tidyverse)
library(elo)
library(ggthemes) # Themes for ggplot2.
library(ggimage) # Supports images in ggplot2.
library(gt) # Create tables.

## Import dataset.

In [3]:
ff_scores <- read.csv(file = "C:/Users/chris/Documents/Datasets/FFL/ff_scores.csv")
ff_scores <- ff_scores %>%
  select(date, team_1, team_2, win) %>%
  arrange(date)

## Generate teams table with initial elo value = 1500.

In [4]:
teams <- data.frame(team = unique(ff_scores$team_1)) %>%
  mutate(elo = 1500)

## Calculate changes in elo rating based on weekly results.

In [5]:
team1_elo_pre_vector <- c()
team2_elo_pre_vector <- c()
team1_elo_post_vector <- c()
team2_elo_post_vector <- c()

for (i in seq_len(nrow(ff_scores))) {
  match <- ff_scores[i, ]
  
  # Pre-match ratings.
  team_1_elo <- subset(teams, team == match$team_1)$elo
  team_2_elo <- subset(teams, team == match$team_2)$elo
  
  # Elo calculation.
  new_elo <- elo.calc(wins.A = match$win,
                      elo.A = team_1_elo,
                      elo.B = team_2_elo,
                      k = 30)
    
  # Post-match ratings  .
  team_1_new_elo <- new_elo[1, 1]
  team_2_new_elo <- new_elo[1, 2]

  team1_elo_pre_vector <- append(team1_elo_pre_vector, team_1_elo)
  team2_elo_pre_vector <- append(team2_elo_pre_vector, team_2_elo)
  team1_elo_post_vector <- append(team1_elo_post_vector, team_1_new_elo)
  team2_elo_post_vector <- append(team2_elo_post_vector, team_2_new_elo)

  # Update team elo ratings table.
  teams <- teams %>%
    mutate(elo = case_when(
        (team == match$team_1) ~ team_1_new_elo,
        (team == match$team_2) ~ team_2_new_elo,
        ((team != match$team_1) & (team != match$team_2)) ~ elo
    ))
}

## Create columns to save pre/post-week elo ratings.

In [6]:
ff_scores$team1_elo_pre <- team1_elo_pre_vector
ff_scores$team2_elo_pre <- team2_elo_pre_vector
ff_scores$team1_elo_post <- team1_elo_post_vector
ff_scores$team2_elo_post <- team2_elo_post_vector

## Elo rating history table.

In [7]:
head(ff_scores)

,date,team_1,team_2,win,team1_elo_pre,team2_elo_pre,team1_elo_post,team2_elo_post
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,2014-01,CHRIS,CARLEN,0,1500,1500,1485.000,1515.000
2,2014-01,ANDREW,KYLE,0,1500,1500,1485.000,1515.000
3,2014-01,STEVE,DREW,1,1500,1500,1515.000,1485.000
4,2014-01,MATT,NATHAN,1,1500,1500,1515.000,1485.000
5,2014-02,MATT,CHRIS,1,1515,1485,1528.708,1471.292
6,2014-02,STEVE,ANDREW,0,1515,1485,1498.708,1501.292


## Current elo rating table.

In [16]:
teams <- teams %>%
  mutate(elo = as.integer(round(elo,0))) %>%
  filter(!(team %in% c("DREW", "NATHAN", "CARLEN"))) %>%
  arrange(-elo)
teams

team,elo
<chr>,<int>
CHRIS,1653
ANDREW,1581
STEVE,1530
SAM,1520
NICK,1514
KYLE,1500
NOAH,1472
JASON,1470
DYLAN,1427


## Create current elo ratings graphic.

In [36]:
teams_gt <- teams %>%
  gt() %>%
  cols_label(team = "",
             elo = "Current Elo") %>%
  cols_align(align = "center",
             columns = c(team, elo)) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Extravaganza FFL, 2021", subtitle = "Current Elo Ratings") %>%
  tab_options(heading.title.font.size = 24,
              heading.border.bottom.color = "#808080") %>%
  tab_style(style = cell_borders(sides = "right",
                                 color = "#808080",
                                 weight = px(5)),
            locations = cells_body(columns = c(team, elo),
                                   rows = everything())) %>%
  tab_style(style = cell_borders(sides = "bottom",
                                 color = "#808080",
                                 weight = px(2)),
            locations = cells_body(columns = everything(),
                                   rows = c(10))) %>%
  tab_style(style = cell_borders(sides = "top",
                                 color = "#808080",
                                 weight = px(2)),
            locations = cells_body(columns = everything(),
                                   rows = 1)) %>%
  data_color(columns = elo,
             colors = scales::col_numeric(palette = c("#FEFE69", "#DDF969", "#A9F36A", "#78EC6C", "#57E86B"),
                                          domain = c(1350, 1700)))

In [37]:
gtsave(teams_gt, "current-elo-ratings.png")

## Export elo rating history table to CSV.

In [9]:
write.csv(ff_scores,"C:\\Users\\chris\\Desktop\\elo_history.csv", row.names = FALSE)